### Introduction
Here is simple approach to rate severity of toxic comments. Find probability of the comment to be toxic one and give the score based on this probability. 

Notebook is build upon this public kernel: https://www.kaggle.com/devkhant24/jigsaw-comment-toxicity-bidirectional-gru. I only changed the model and added some comments

In [ ]:
import math
import os
import random
import numpy as np
import pandas as pd
import re
import unidecode
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from imblearn.under_sampling import RandomUnderSampler
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, Dropout, GRU, Embedding, LSTM, Bidirectional, Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Concatenate#, LSTM, GRU

### Hyperparameters and Helper Functions
All hyperparameters need to be tuned in order to achieve the best result from the mode.

In [ ]:
voc_size = 50000   # number of words in vocabulary to make embedding
max_sequence_length = 300 # take maximum 300 characters in the input sequence
embedding_dim = 128 # create the word embeddings in this dimension. Any text input willl be mapped into vector of lendth 128

EPOCHS = 70 
BATCH_SIZE = 2048 
LEARNING_RATE = 0.001
FOLDS = 4 
VERBOSE = 0

In [ ]:
# Cleaning the text  from unnecessary characters
def clean_data(data):
    final = []
    for sent in data:
        sent = sent.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')
        soup = BeautifulSoup(sent, "html.parser")
        sent = soup.get_text(separator=" ")
        remove_https = re.sub(r'http\S+', '', sent)
        sent = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
        sent = unidecode.unidecode(sent)
        sent = sent.lower()
        sent = re.sub(r"[^a-zA-Z0-9:$-,()%.?!]+", ' ', sent) 
        sent = re.sub(r"[:$-,()%.?!]+", ' ',sent)
        stoplist = stopwords.words("english")
        sent = [word for word in word_tokenize(sent) if word not in stoplist]
        sent = " ".join(sent)
        final.append(sent)
    return final

# Return the probability from output number
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

# Set seed to get the same output everytime
def seed_everything(SEED=13):
    np.random.seed(SEED)
    random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
    os.environ['PYTHONHASHSEED'] = str(SEED)

seed_everything()

### Preprocessing
Take dataset from toxic comment classification challenge that classifies comments into 1 of 7 categories (toxic, severe_toxic, obscene, threat, insult, identity_hate). Seventh category is when all of them are zeros, means the comment is not toxic

In [ ]:
train_prev_comp = "../input/toxic-comment/jigsaw-toxic-comment-train.csv"
test_cur_comp = "../input/jigsaw-toxic-severity-rating/comments_to_score.csv"

Then combine all 6 categories into one. So now dataset just shows if the comment is toxic or not. Therefore we simply have binary classification problem

In [ ]:
# Reading train file from previous competition
df = pd.read_csv(train_prev_comp)

df["y"] = (df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].sum(axis=1) > 0).astype(int)
df.drop(["id","toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"], axis=1, inplace = True)
df.head()

In [ ]:
df["y"].value_counts()

Dataset is imbalanced as there are more 0 values than 1. Training model on such data will give biased result. To solve it, undersampling is used. It randomly drops some data with zero values and makes values to be equal

In [ ]:
X = np.array(df["comment_text"].values)
X = X.reshape(-1,1)
y = np.array(df["y"].values)
rus = RandomUnderSampler(random_state=0)
train, target = rus.fit_resample(X, y)

train = train.flatten()
df = pd.DataFrame()
df["text"] = train
df["target"] = target

# Now its balanced
df["target"].value_counts()

In [ ]:
# Cleaning the data
df["text"] = clean_data(df["text"])

### Model and Training
We define LSTM network that has feedback connections and therefore can find relationship of samples in the sequential input. It is better than GRU as it remembers more information, however takes more time to be trained

In [ ]:
# Defining sequential model with LSTM units. 
def lstm_model():
    x_input = Input(shape=(max_sequence_length))
    x = Embedding(voc_size, embedding_dim, input_length = max_sequence_length)(x_input)
    x = LSTM(256, return_sequences=True)(x)
    x = Dropout(0.1)(x)
    x = LSTM(128, return_sequences=True)(x)
    x = Dropout(0.1)(x)
    
    y = LSTM(64, return_sequences=False)(x)
    y = Dropout(0.1)(y)
    z = GRU(64, return_sequences=False)(x)
    z = Dropout(0.1)(z)
    
    x = Concatenate()([y, z])
    x = Dense(units=32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.25)(x)
    
    x_output = Dense(units=1, activation='sigmoid')(x)
    
    model = Model(inputs=x_input, outputs=x_output, name='LSTM_Model')
    return model

In [ ]:
model = lstm_model()
model.summary()

In [ ]:
plot_model(
    model, 
    to_file='LSTM.png', 
    show_shapes=True,
    show_layer_names=True
)

Model is trained for several folds that helps to use data efficiently. Every time 3/4th dataset will be used as training data and remaining as validation. 4 models will be trained and their predictions averaged

In [ ]:
preds_valid_f = {}
preds_test = []
total_auc = []
f_scores = []
counter = 0

kf = StratifiedKFold(n_splits=FOLDS,random_state=0,shuffle=True)

train = df["text"]
target = df["target"]

for fold,(train_index, valid_index) in enumerate(kf.split(train, target)):

    counter +=1
    
    X_train,X_valid = train.loc[train_index], train.loc[valid_index]
    y_train,y_valid = target.loc[train_index], target.loc[valid_index]
  
    # Preprocessing
    tokenizer = Tokenizer(num_words = voc_size)
    tokenizer.fit_on_texts(X_train.values)
    X_train = tokenizer.texts_to_sequences(X_train.values)
    X_train = pad_sequences(X_train, maxlen = max_sequence_length)
    #print(X_train.shape)
 
    X_valid = tokenizer.texts_to_sequences(X_valid.values)
    X_valid = pad_sequences(X_valid, maxlen = max_sequence_length)
    
    model = lstm_model()
    model.compile(
        optimizer= Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['AUC'],
    )
    
    lr = ReduceLROnPlateau(monitor="val_loss", factor=0.25, 
                           patience=5, verbose=VERBOSE, mode='min')
    
    # checkpoint save the model witg best validation accuracy
    chk_point = ModelCheckpoint(f'./Model_{counter}C.h5', 
                                monitor='val_loss', verbose=VERBOSE, 
                                save_best_only=True, mode='min')
    es = EarlyStopping(
        patience=10,
        min_delta=0,
        monitor='val_loss',
        restore_best_weights=True,
        verbose=0,
        mode='min', 
        baseline=None,
    )

    history = model.fit(  X_train, y_train,
                validation_data = (X_valid, y_valid),
                batch_size = BATCH_SIZE, 
                epochs = EPOCHS,
                callbacks = [es, lr, chk_point],
                shuffle = True,
                verbose = 0
              )
    
    model = load_model(f'./Model_{counter}C.h5')

    #  model predicts wheither test dataset is toxic or not.
    # Probability is then multiplied by 100 and we get score
    test = pd.read_csv(test_cur_comp)
    test["text"] = clean_data(test["text"])
    x_test = tokenizer.texts_to_sequences(test["text"].values)
    x_test = pad_sequences(x_test, maxlen = max_sequence_length)
    pred = model.predict(x_test)
    pred = [sigmoid(x) * 100 for x in pred]
    
    preds_test.append(pred)

Note that such approach is very naive as it treats any toxic class equally while identity_hate for example should get larger score than obscene and severe_toxic comment is worse than toxic. It is better to classify the comment and assign different score for each class.

### Submission

In [ ]:
final_pred = np.mean(preds_test, axis = 0)

In [ ]:
# Making submission file
final = pd.DataFrame()
final["comment_id"] = test["comment_id"]
final["score"] = pred
final.to_csv("submission.csv", index=False)